In [1]:
import os, glob
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import torch

from ultralytics import YOLO
import easyocr
import utils

In [2]:
if "temp" not in os.listdir():
    os.makedirs("./temp")

<h3 style="color:orange">Hyperparameters</h3>

In [ ]:
DEVICE="cuda" if torch.cuda.is_available() else "cpu"
DF=pd.DataFrame(columns=["Frame No.","Lincenc Plate bbox","timestamp","text","score","valid"])

print(f"{DEVICE=}")

vid_dirs=["./dataset/video/demo.mp4","./dataset/video/IND_demo.mp4"]
vid_dir=vid_dirs[1]

vid=cv2.VideoCapture(vid_dir)

# video metadata
frame_count=int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
W=int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
H=int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS=int(vid.get(cv2.CAP_PROP_FPS))

print(f"frame count: {frame_count}\nframe width: {W}\nframe hight: {H}\nFPS: {FPS}")


DEVICE='cuda'
frame count: 631
frame width: 1114
frame hight: 720
FPS: 30


<h3 style="color:orange">Load Model</h3>

In [4]:
model_weight=[path for path in glob.glob("./Model/weights/*.pt")]
print(model_weight)
model=YOLO(model_weight[1]).to(DEVICE)

matter_tracker=utils.sort.Sort()   #use for tracting object in continues frames

print(model)

['./Model/weights\\yolov8n.pt', './Model/weights\\YOLOv8_LPD_largeset.pt', './Model/weights\\YOLOv8_LPD_roboflow.pt']
YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1,

<h3 style="color:orange">model infernce</h3>

In [ ]:
results={}

if "video" not in os.listdir("./temp/"): os.makedirs("./temp/video")
# if "true_frame" not in os.listdir("./temp/"): os.makedirs("./temp/true_frame")
# video save
out_mp4=cv2.VideoWriter(f"./temp/video/temp_{os.path.splitext(os.path.basename(vid_dir))[0]}.mp4",cv2.VideoWriter_fourcc(*"mp4v"),FPS,(W,H))

LIMIT=50

ret,frame_num=True,-1
while(ret):
    frame_num+=1
    ret,frame=vid.read()
    # cv2.imwrite(f"./temp/true_frame/{frame_num}.png",frame)
    
    print(f"Frame no. {frame_num}")
    if(ret):

        # if(frame_num>LIMIT): break     #debug
        
        results[frame_num]={}

        # detect numberplate
        detections=model(frame)[0]
        detection_list=[]
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=detection
            if(score>=0.5):
                # print([int(x1),int(y1),int(x2),int(y2),score])
                detection_list.append([int(x1),int(y1),int(x2),int(y2),score])
        print("\n")
    

        ## track vehicle 
        # track_id=matter_tracker.update(np.asarray(detection_list))
        # print(track_id)

        # crop Lincence plate
        crop_license_plate=frame[int(y1):int(y2),int(x1):int(x2),:]
        # print(crop_license_plate.shape)
        if "crop" not in os.listdir("./temp/"): os.makedirs("./temp/crop")
        cv2.imwrite(f"./temp/crop/{frame_num}.png",crop_license_plate)    ## DEBUG

        ## correct one
        text,score,Flag=utils.utils.read_license_plate_video(crop_license_plate)
        
        time_stamp=frame_num/FPS

        # add Data to CSV
        DF.loc[len(DF)]=[frame_num+1,(int(x1),int(y1),int(x2),int(y2)),time_stamp,text,score,Flag]

        if(len(text)==0):
            out_mp4.write(frame)        # to add blank predition files
            continue

        if "ans_img" not in os.listdir("./temp/"): os.makedirs("./temp/ans_img")

        # add color base on correct OCR identification
        color=(0,255,0) if Flag else (0,0,255)
        # bbox
        cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),(int(y2))),color,3)
        # text
        cv2.putText(frame,text,(int(x1),int(y1)-3),cv2.FONT_HERSHEY_PLAIN,2,color,2,cv2.LINE_AA)
        cv2.imwrite(f"./temp/ans_img/{os.path.splitext(os.path.basename(vid_dir))[0]}_{frame_num}.png",frame)    ## DEBUG

    ##  save video
    out_mp4.write(frame)

# clear all video caps
out_mp4.release()
cv2.destroyAllWindows()

Frame no. 0

0: 416x640 2 License Platess, 39.8ms
Speed: 2.8ms preprocess, 39.8ms inference, 76.5ms postprocess per image at shape (1, 3, 416, 640)


Frame no. 1

0: 416x640 2 License Platess, 6.4ms
Speed: 1.9ms preprocess, 6.4ms inference, 1.6ms postprocess per image at shape (1, 3, 416, 640)


Frame no. 2

0: 416x640 2 License Platess, 7.0ms
Speed: 1.6ms preprocess, 7.0ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 640)


Frame no. 3

0: 416x640 3 License Platess, 6.3ms
Speed: 1.7ms preprocess, 6.3ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)


Frame no. 4

0: 416x640 3 License Platess, 5.9ms
Speed: 1.5ms preprocess, 5.9ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)


Frame no. 5

0: 416x640 3 License Platess, 5.8ms
Speed: 1.4ms preprocess, 5.8ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)


Frame no. 6

0: 416x640 3 License Platess, 6.1ms
Speed: 2.3ms preprocess, 6.1ms inference, 1.3ms postprocess per im

<h3 style="color:orange">CSV</h3>

In [6]:
# DF.dropna(subset=["text"],inplace=True)
DF=DF[DF["text"]!=""]          # drop empty text column

# remove consecative duplicates 
DF=DF[DF["text"]!=DF["text"].shift()]

# save this DF
if "csv" not in os.listdir("./temp/"): os.makedirs("./temp/csv")
DF.to_csv(f"./temp/csv/{os.path.splitext(os.path.basename(vid_dir))[0]}.csv",index=False)

In [7]:
DF

,Frame No.,Lincenc Plate bbox,timestamp,text,score,valid
0,1,"(604, 525, 783, 582)",0.000000,RIBF,0.202233,False
1,2,"(632, 0, 732, 96)",0.033333,6 0,0.999492,False
8,9,"(487, 498, 674, 552)",0.266667,REB } J,0.074095,False
13,14,"(445, 488, 618, 525)",0.433333,R - 183 - JE,0.434172,False
14,15,"(423, 477, 606, 525)",0.466667,R - 183 - JF,0.571693,False
...,...,...,...,...,...,...
580,581,"(448, 461, 611, 497)",19.333333,Rh - 3,0.020778,False
581,582,"(448, 461, 611, 497)",19.366667,K l 3,0.028608,False
582,583,"(401, 452, 557, 488)",19.400000,ILC 3,0.008008,False
583,584,"(370, 451, 528, 488)",19.433333,7 3,0.261904,False
